# ECDS Shock Index Overview

This notebook introduces the initial data sources and formula used to compute an ECDS Shock Index for Medicare Advantage Stars measures.

## Data Sources

- NCQA HEDIS ECDS results (example extract in `data/raw/example_ncqa_ecds.csv`)
- CMS Stars measure definitions and weights (example extract in `data/raw/example_cms_measure_weights.csv`)

## Index Formula

\[\text{ShockIndex} = \alpha\cdot CCS + \beta\cdot EAV + \gamma\cdot CPR + \delta\cdot WM\]\n
Default weights in this repository: \n- \alpha = 0.35\n- \beta = 0.25\n- \gamma = 0.20\n- \delta = 0.20

In [ ]:
from ecds_shock_index import ShockIndexCalculator, ccs_score, eav_score, cpr_score, wm_score, classify_risk

calculator = ShockIndexCalculator()

# Single-measure example
ccs = ccs_score(0.82, 0.90)
eav = eav_score(1.15)
cpr = cpr_score(0.20)
wm = wm_score(3, max_weight=5)

index = calculator.calculate(ccs=ccs, eav=eav, cpr=cpr, wm=wm)
tier = classify_risk(index)
print(f"Shock Index: {index:.4f}  ({tier} risk)")

## Batch Scoring

Load the example CSVs, score every measure, and aggregate to a contract-level summary.

In [ ]:
from ecds_shock_index.data_loader import load_ncqa_ecds, load_cms_measure_weights, merge_ecds_and_weights

# Load and merge data
ecds = load_ncqa_ecds("../data/raw/example_ncqa_ecds.csv")
weights = load_cms_measure_weights("../data/raw/example_cms_measure_weights.csv")
merged = merge_ecds_and_weights(ecds, weights)

# Score all measures at once
scored = calculator.score_dataframe(merged)
scored[["measure_id", "ccs", "eav", "cpr", "wm", "shock_index", "risk_tier"]]

## Contract-Level Aggregation

Combine individual measure scores into a single contract-level risk summary.

In [ ]:
# Contract-level summary (weighted by CMS measure weight)
summary = calculator.aggregate_contract(scored)
for key, val in summary.items():
    print(f"  {key}: {val}")